## imports

In [1]:
import pandas as pd
import os
import json

In [2]:
import re
import copy

In [3]:
#find "#old style" for other type of texts separators

## READ THE FILE

In [259]:
path = 'lighttag_annotations6.json'
filename, file_extension = os.path.splitext(path)
print(filename)
print(file_extension)

lighttag_annotations6
.json


In [260]:
raw_data = []
for line in open(path, 'r', encoding='utf-8'):
    raw_data.append(json.loads(line))
    annotations_and_examples = []
    relations = []
    #у нас джейсон в одну линию записан, поэтому по сути annannotations_and_examples и relations не обнуляются, и накапливают всю
    #инфу, которую надо
    for d in raw_data:
        for k, v in d.items():
            if k == 'annotations_and_examples':
                annotations_and_examples.append(v)
            if k == 'relations':
                relations.append(v)
                
#we have no relations in our task, so we need not them to append, so we need not the_list


In [261]:
#annotations_and_examples contains:

# content
# metadata
# annotations 
# classifications


In [262]:
raw_data = []
for line in annotations_and_examples[0]:
    raw_data.append((line))
    content = []
    metadata = []
    annotations = []
    classifications = []
    for d in raw_data:
        for k, v in d.items():
            if k == 'content':
                content.append(v)
            if k == 'metadata':
                metadata.append(v)
            if k == 'annotations':
                annotations.append(v)
            if k == 'classifications':
                classifications.append(v)

In [263]:
#size = 1
size = len(content)
print(size)

1


In [264]:
annotations[0][0]

{'start': 25, 'end': 48, 'tag': 'м'}

In [265]:
annotations[size-1][0]
#annotations[3][0]

{'start': 25, 'end': 48, 'tag': 'м'}

In [266]:
content[size-1]

"'\\n############\\n2861,Суд в Новгородской области арестовал начальника управления экономической безопасности и противодействия коррупции ( УЭБиПК ) УМВД России по региону за присвоение выплат ветеранам боевых действий на протяжении десяти лет . Об этом сообщается на сайте следственного управления СКР по Новгородской области . Суд отправил сотрудника полиции под домашний арест . Он обвиняется по ч . 3 ст . 159 . 2 УК РФ ( мошенничество при получении выплат ) и ч . 1 ст . 285 УК РФ ( злоупотребление должностными полномочиями ) . По версии следствия , обвиняемый в течении десяти лет незаконно получал ежемесячные денежные выплаты , полагающиеся ветеранам боевых действий . В результате им были похищены бюджетные денежные средства в размере около 250 тыс . руб . , — говорится в сообщении . Кроме того , как отметили в СК , полицейскому вменяется злоупотребление должностными полномочиями при выделении ему на безвозмездной основе участка земли на озере Селигер . ,В Новгородской области обвини

## READ THE DATA

## TAGS

In [267]:
starts = []
ends = []
tags = []

for i in range(len(annotations)):
    raw_data = []
    for line in annotations[i]:
        raw_data.append((line))
        st = []
        en = []
        tg = []
        for d in raw_data:
            for k, v in d.items():
                if k == 'start':
                    st.append(v)
                if k == 'end':
                    en.append(v)
                if k == 'tag':
                    tg.append(v)
    starts.append(st)
    ends.append(en)
    tags.append(tg)

len(starts)

1

In [268]:
len(starts[size-1])

59

In [269]:
content[size-1][starts[size-1][15]:ends[size-1][15]]

'Татарстану '

In [270]:
len(tags[0])

59

###  пустышка, заполняем нулями

In [271]:
tagged_content = copy.copy(content)

In [272]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, size):
    print(len(tagged_content[i])==len(content[i]))

True 

True


In [273]:
for j in range(0, size):
    list_content = list(tagged_content[j])

    for i in range(0, len(list_content)):
        #old style
        #if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != '\n':
        if list_content[i] != ' ' and list_content[i] != '#' and list_content[i] != 'n' and list_content[i] != '\\':
            list_content[i] = 'O'

    tagged_content[j] = "".join(list_content)

In [274]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, size):
    print(len(tagged_content[i])==len(content[i]))

True 

True


## Избегаем схлопывания многословных тегов: все, что в промежутке тега, заполняем его индикатором, но если этот промежуток совпал с пробелом в оригинальном контенте - тут тоже пробел

In [275]:
for q in range(0, size):
    list_content = list(tagged_content[q])

    for i in range(0, len(tags[q])):
        for j in range(starts[q][i], ends[q][i]):
            if content[q][j] == ' ':
                list_content[j] = ' '
            else:
                list_content[j] = tags[q][i]

    tagged_content[q] = "".join(list_content)

In [276]:
print(len(tagged_content) == len(content), '\n')

for i in range(0, size):
    print(len(tagged_content[i])==len(content[i]))

True 

True


## хлопаем получившиеся посхедовательности до одного символа

In [277]:
for i in range(0, len(tagged_content)):
    tagged_content[i] = re.sub(r'O+', 'O', tagged_content[i])
    tagged_content[i] = re.sub(r'ж+', 'ж', tagged_content[i])
    tagged_content[i] = re.sub(r'п+', 'п', tagged_content[i])
    tagged_content[i] = re.sub(r'д+', 'д', tagged_content[i])
    tagged_content[i] = re.sub(r'м+', 'м', tagged_content[i])

In [278]:
for i in range(0, size):
    bull = (len(tagged_content[i].split(' ')) == len(content[i].split(' ')))
    print(bull, '\n')

True 



## TAGS AND TEXT BOTH

## Разделяем на тексты и теги к текстам

In [279]:
list_of_tags = []
sub_list_of_tags = []
for i in range(0, len(tagged_content)):
    sub_list_of_tags.append(tagged_content[i].split('\\n############\\n'))
    #old style
    #sub_list_of_tags.append(tagged_content[i].split('\n############\n'))

for i in range(0, size):
    list_of_tags+=sub_list_of_tags[i]

In [280]:
list_of_texts = []
sub_list_of_texts = []
for i in range(0, len(content)):
    sub_list_of_texts.append(content[i].split('\\n############\\n'))
    #old style
    #sub_list_of_texts.append(content[i].split('\n############\n'))

    
for i in range(0, size):
    list_of_texts+=sub_list_of_texts[i]

In [281]:
print(len(list_of_texts))
print(len(list_of_tags))
#список со всеми текстами и список со всеми текстами тегов, элемент - один текст строкой или один текст тегов строкой

8
8


In [282]:
list_of_text_tokens = []
for i in range(0, len(list_of_texts)):
    list_of_text_tokens.append([list_of_texts[i].split(' ')])

In [283]:
list_of_tags_tokens = []
for i in range(0, len(list_of_tags)):
    list_of_tags_tokens.append([list_of_tags[i].split(' ')])

In [284]:
# в целом те же самые списки, только теперь элемент - список токенов или список тегов (строки разбили на отдельные элементы)

## Проверка совпадения количество тегов - токенов

In [285]:
for i in range(0, len(list_of_text_tokens)):
    if len(list_of_tags_tokens[i][0])!=len(list_of_text_tokens[i][0]):
        print('idx ', i)
        print(len(list_of_tags_tokens[i][0]))
        print(len(list_of_text_tokens[i][0]))
        print('\n')

## Проверка соответствия тегов токенам

In [286]:
#проверяем по-токеново idx-й текст
idx = 4
for i in range(0, len((list_of_text_tokens[idx][0]))):
    if list_of_tags_tokens[idx][0][i] !='O':
        print('слово ',list_of_text_tokens[idx][0][i])
        print('тег ',list_of_tags_tokens[idx][0][i])
        print('\n')

слово  Татарстана
тег  м


слово  Владимиру
тег  п


слово  Швецову
тег  п


слово  Владимир
тег  п


слово  Швецов
тег  п


слово  Татарстан
тег  м


слово  Владимира
тег  п


слово  Швецова
тег  п


слово  Татарстан
тег  м


слово  Татарстану
тег  м


слово  Владимира
тег  п


слово  Швецова
тег  п


слово  Владимир
тег  п


слово  Швецов-младший
тег  пO


слово  Наталья
тег  п


слово  Устинова
тег  п


слово  Швецов-младший
тег  пO


слово  Владимира
тег  п


слово  Швецова
тег  п


слово  Владимира
тег  п


слово  Швецова
тег  п


слово  Швецову
тег  п




## Переворачиваем в нормальные теги

In [287]:
final_list_of_tags_tokens = []
for i in range(0, len(list_of_tags_tokens)):
    sublist = []
    for j in range(0, len(list_of_tags_tokens[i][0])):
        if list_of_tags_tokens[i][0][j] == 'п':
            sublist.append('OFF')
        elif list_of_tags_tokens[i][0][j] == 'ж':
            sublist.append('SUF')
        elif list_of_tags_tokens[i][0][j] == 'д':
            sublist.append('DAT')
        elif list_of_tags_tokens[i][0][j] == 'м':
            sublist.append('LOC')
        else:
            sublist.append('O')
    final_list_of_tags_tokens.append(sublist)
    

In [288]:
len(final_list_of_tags_tokens)

8

In [289]:
len(final_list_of_tags_tokens[0])

1

In [290]:
len(list_of_tags_tokens[0][0])

1

## Заносим все в один датафрейм, подтягиваем столб с оригинальной датой, сохр csv

In [291]:
final = []
for i in range(0, len(list_of_text_tokens)):
    final.append([list_of_text_tokens[i][0], final_list_of_tags_tokens[i]])

In [292]:
str_final = []
for i in range(1, len(final)):
    str_text = ' '.join(final[i][0])
    str_tags = ' '.join(final[i][1])
    str_final.append([str_tags, str_text])
str_final[0][0]

'O LOC LOC LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O LOC LOC LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O LOC LOC O O O O O O O'

### удаляем номер текста и его айдишник (и внешние кавычки) - ужасные артефакты

In [293]:
len(str_final[0][1].split(" "))

161

In [294]:
len(str_final[0][0].split(" "))

161

In [295]:
#old style
#len(''.join(str_final[0][1].split(",")[1:-1]).split(' '))
len(','.join(str_final[0][1].split(",")[1:-2]).split(' '))

160

In [296]:
str_final[30][1].split(' ')

IndexError: list index out of range

In [297]:
','.join(str_final[30][1].split(",")[1:-2]).split(' ')

IndexError: list index out of range

In [298]:
for i in range(0, len(str_final)):
    str_final[i][1] = ','.join(str_final[i][1].split(",")[1:-2])
    #эта строка появилась потому что при моем стиле формирования текстов на разметку в конце каждого разделителя выползает кавычка, которой присваивается собственный тег
    #вот он-то сейчас и удалится
    str_final[i][0] = ' '.join(str_final[i][0].split(' ')[:-1])

In [299]:
#old style
#for i in range(0, len(str_final)):
#    str_final[i][1] = ','.join(str_final[i][1].split(",")[1:-1])

In [300]:
#old style
#for i in range(0, len(str_final)):
#    str_final[i][1] = str_final[i][1].replace('"', '')

In [301]:
len(str_final[0][0].split(' '))

160

In [302]:
len(str_final[0][1].split(' '))

160

## все, датафрейм

In [303]:
df = pd.DataFrame(str_final, columns=['tags', 'texts'])
df.head()

,tags,texts
0,O LOC LOC LOC O O O O O O O O O O O O O O O O ...,Суд в Новгородской области арестовал начальник...
1,O O O O O O O O O O O O O O O O O O O O O O O ...,Бывшего начальника якутского авиационного спор...
2,O LOC O O O O O O O O O O O O O O O O O O O O ...,В Карелии возбуждено уголовное дело по факту з...
3,O O O LOC O O O O O OFF OFF O O O O O O O O O ...,Бывшему заместителю премьер-министра Татарстан...
4,O O LOC O O O O O O O O O O O O O OFF OFF OFF ...,Басманный суд Москвы принял решение продлить д...


In [304]:
df['texts'][0]

'Суд в Новгородской области арестовал начальника управления экономической безопасности и противодействия коррупции ( УЭБиПК ) УМВД России по региону за присвоение выплат ветеранам боевых действий на протяжении десяти лет . Об этом сообщается на сайте следственного управления СКР по Новгородской области . Суд отправил сотрудника полиции под домашний арест . Он обвиняется по ч . 3 ст . 159 . 2 УК РФ ( мошенничество при получении выплат ) и ч . 1 ст . 285 УК РФ ( злоупотребление должностными полномочиями ) . По версии следствия , обвиняемый в течении десяти лет незаконно получал ежемесячные денежные выплаты , полагающиеся ветеранам боевых действий . В результате им были похищены бюджетные денежные средства в размере около 250 тыс . руб . , — говорится в сообщении . Кроме того , как отметили в СК , полицейскому вменяется злоупотребление должностными полномочиями при выделении ему на безвозмездной основе участка земли на озере Селигер . ,В Новгородской области обвинили полицейского в присво

In [305]:
for i in range(0, len(df)):
    if len(df['texts'][i].split(' '))!=len(df['tags'][i].split(' ')):
        print('idx ', i)
        print(df['texts'][i])
        print(len(df['texts'][i].split(' ')))
        print('\n')
        print(df['tags'][i])
        print(len(df['tags'][i].split(' ')))
        print('\n')

idx  6
Полиция Камбоджи задержала в столице страны Пномпене 119 китайцев по подозрению в мошенничестве . Об этом сообщает Reuters . Подозреваемых задержали по двум адресам 22 апреля . Представители силовиков уточнили , что это было сделано по запросу китайских спецслужб . Детали мошеннической схемы агентство не уточняет . Reuters отмечает только , что жертвами мошенничества становились граждане КНР и что расследование длилось несколько месяцев . По данным иммиграционной полиции , китайцы приехали в Камбоджу по туристическим визам . В августе 2017 года Reuters сообщало об аресте 400 граждан Китая и Тайваня по аналогичному обвинению . 
95


O LOC O O O O LOC O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O O LOC O O O O O O O O O O O O O O LOC O LOC O O O O O LOC O O O O
100




In [306]:
df.to_csv(filename+'_parsed_data_str.csv', index = False)

### проверяем формат строк и все такое при подгрузке сформированного файла обратно

In [307]:
df1 = pd.read_csv(filename+'_parsed_data_str.csv')
df1.head()

,tags,texts
0,O LOC LOC LOC O O O O O O O O O O O O O O O O ...,Суд в Новгородской области арестовал начальник...
1,O O O O O O O O O O O O O O O O O O O O O O O ...,Бывшего начальника якутского авиационного спор...
2,O LOC O O O O O O O O O O O O O O O O O O O O ...,В Карелии возбуждено уголовное дело по факту з...
3,O O O LOC O O O O O OFF OFF O O O O O O O O O ...,Бывшему заместителю премьер-министра Татарстан...
4,O O LOC O O O O O O O O O O O O O OFF OFF OFF ...,Басманный суд Москвы принял решение продлить д...


In [308]:
list_of_tags = []
for i in range(0, len(df1['tags'])):
    if i!=49:
        list_of_tags.append(df1['tags'][i].split(' '))

list_of_texts = []
for i in range(0, len(df1['texts'])):
    if i!=49:
        list_of_texts.append(df1['texts'][i].split(' '))


In [309]:
for i in range(0, len(list_of_texts)):
    if len(list_of_tags[i])!=len(list_of_texts[i]):
        print('i: ',i)
        print('diff: ',len(list_of_texts[i])- len(list_of_tags[i]))

i:  6
diff:  -5


## MAKING FILES FOR DEEP PAVLOV TRAIN NER

In [56]:
file = open('train.txt','w', encoding='utf-8') 
for i in range(0, int(0.8*len(list_of_texts))):
    file.write('-DOCSTART-' + '\t'+'O'+'\n')
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
        if list_of_texts[i][j] == '.':
            file.write('\n')
            file.write('\n')
file.close()

In [62]:
file = open('test.txt','w', encoding='utf-8') 
for i in range(int(0.8*len(list_of_texts)), int(0.9*len(list_of_texts))):
    file.write('-DOCSTART-' + '\t'+'O'+'\n')
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
        if list_of_texts[i][j] == '.':
            file.write('\n')
            file.write('\n')
file.close()

In [63]:
file = open('valid.txt','w', encoding='utf-8') 
for i in range(int(0.9*len(list_of_texts)), len(list_of_texts)):
    file.write('-DOCSTART-' + '\t'+'O'+'\n')
    for j in range(0, len(list_of_texts[i])):
        line = list_of_texts[i][j] + '\t' + list_of_tags[i][j] + '\n'
        file.write(line)
        if list_of_texts[i][j] == '.':
            file.write('\n')
            file.write('\n')
file.close()